In [22]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import glob
import os
import re

import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import en_core_web_sm
import re
import datetime
from afinn import Afinn
tqdm.pandas()

nlp = spacy.load("en_core_web_sm")

In [2]:
df_movie = pd.read_csv('/Users/yli17/Downloads/movie_data_2015_2020.csv', error_bad_lines=False)
df_movie.head()

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,...,ordering_y,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,0,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",...,10.0,nm6618222,producer,producer,NaN,Lindsey Weber,NaN,NaN,producer,"tt4530422,tt2660888,tt2548396,tt1179933"
1,1,tt4530422,movie,Overlord,Overlord,0.0,2018.0,NaN,110.0,"Action,Horror,Sci-Fi",...,9.0,nm6618222,producer,producer,NaN,Lindsey Weber,NaN,NaN,producer,"tt4530422,tt2660888,tt2548396,tt1179933"
2,2,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",...,1.0,nm0000422,actor,NaN,"[""Howard""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"
3,3,tt2406566,movie,Atomic Blonde,Atomic Blonde,0.0,2017.0,NaN,115.0,"Action,Thriller",...,3.0,nm0000422,actor,NaN,"[""Emmett Kurzfeld""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"
4,4,tt5968394,movie,Captive State,Captive State,0.0,2019.0,NaN,109.0,"Action,Horror,Sci-Fi",...,1.0,nm0000422,actor,NaN,"[""William Mulligan""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"


In [3]:
df_movie['startYear'] = df_movie['startYear'].astype(int)

df_movie = df_movie[df_movie["startYear"]==2015]

In [4]:
moviename = df_movie[df_movie["primaryTitle"].str.contains("The Big Short",na=False)]
moviename

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,...,ordering_y,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
401,401,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,10.0,nm0586969,producer,producer,NaN,Arnon Milchan,1944.0,NaN,"producer,miscellaneous,actor","tt1663202,tt2024544,tt0119488,tt2562232"
797,797,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,4.0,nm0000093,actor,NaN,"[""Ben Rickert""]",Brad Pitt,1963.0,NaN,"actor,producer,soundtrack","tt1210166,tt0114746,tt2935510,tt0356910"
816,816,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,8.0,nm0306890,producer,producer,NaN,Dede Gardner,NaN,NaN,"producer,location_management","tt4975722,tt1020072,tt2024544,tt7125860"
832,832,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,9.0,nm1250070,producer,producer,NaN,Jeremy Kleiner,NaN,NaN,"producer,miscellaneous","tt1020072,tt7125860,tt4975722,tt2024544"
1499,1499,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,5.0,nm0570912,director,NaN,NaN,Adam McKay,1968.0,NaN,"producer,writer,director","tt1596363,tt6266538,tt1386588,tt0478970"
1974,1974,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,2.0,nm0136797,actor,NaN,"[""Mark Baum""]",Steve Carell,1962.0,NaN,"actor,producer,writer","tt0425061,tt0386676,tt1100089,tt0405422"
2406,2406,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,3.0,nm0331516,actor,NaN,"[""Jared Vennett""]",Ryan Gosling,1980.0,NaN,"actor,soundtrack,producer","tt1856101,tt3783958,tt1120985,tt0332280"
2543,2543,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,6.0,nm1017488,writer,screenplay by,NaN,Charles Randolph,NaN,NaN,"writer,producer,executive","tt6394270,tt0289992,tt1596363,tt0758752"
4107,4107,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,1.0,nm0000288,actor,NaN,"[""Michael Burry""]",Christian Bale,1974.0,NaN,"actor,editorial_department,producer","tt0468569,tt1800241,tt0372784,tt0144084"
8533,8533,tt1596363,movie,The Big Short,The Big Short,0.0,2015,NaN,130.0,"Biography,Comedy,Drama",...,7.0,nm1034122,writer,based upon the book by,NaN,Michael Lewis,1960.0,NaN,writer,"tt1210166,tt3657682,tt0878804,tt1596363"


In [5]:
quotes = pd.read_csv('/Users/yli17/Downloads/movie_2015_crew_quotes.csv', error_bad_lines=False)
quotes.head()

,Unnamed: 0,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,23,2015-03-25-025189,how were you able to get that cat to do what y...,Cate Blanchett,['Q80966'],2015-03-25 21:26:54,1,"[['Cate Blanchett', '0.6709'], ['None', '0.329...",['http://www.pedestrian.tv/news/arts-and-cultu...,E
1,55,2015-11-10-031159,However the down side is we are very short of ...,Mark Williams,"['Q13427287', 'Q16192990', 'Q16194404', 'Q1861...",2015-11-10 12:21:00,1,"[['Mark Williams', '0.8594'], ['None', '0.1406']]",['http://www.elystandard.co.uk/help_needed_wit...,E
2,98,2015-09-08-000937,365 degrees is above the glass transition phas...,Tim Moore,"['Q16123784', 'Q16732824', 'Q7804002', 'Q78040...",2015-09-08 12:47:51,1,"[['Tim Moore', '0.8868'], ['None', '0.1132']]",['http://www.thefashionspot.com/beauty/637227-...,E
3,131,2015-01-16-022448,I am also one of the males in this male-domina...,Taapsee Pannu,['Q7672815'],2015-01-16 09:24:12,1,"[['Taapsee Pannu', '0.8301'], ['None', '0.0826...",['http://www.glamsham.com/movies/scoops/15/jan...,E
4,151,2015-04-12-012121,I am glad he's [ Sanjay Leela Bhansali ] chose...,Deepika Padukone,['Q159178'],2015-04-12 06:48:34,1,"[['Deepika Padukone', '0.6817'], ['Sonakshi Si...",['http://us.india.com/showbiz/omg-priyanka-cho...,E


In [6]:
quotes = quotes[quotes["quotation"].str.contains("The Big Short|the big short",na=False)]
quotes["primaryTitle"] = "The Big Short"
quotes.head()

,Unnamed: 0,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,primaryTitle
15336,638594,2015-12-21-033867,"I thought, Ira Glass is no dope-I'll read that...",Adam McKay,"['Q350405', 'Q41538972']",2015-12-21 08:00:00,1,"[['Adam McKay', '0.7979'], ['None', '0.1463'],...",['http://www.kansascity.com/entertainment/movi...,E,The Big Short
18502,767877,2015-12-09-132127,"Well, definitely, Paul [ Rudd ], and we've got...",Peyton Reed,['Q944763'],2015-12-09 03:21:58,2,"[['Peyton Reed', '0.6529'], ['None', '0.1452']...",['http://movieweb.com/ant-man-2-wasp-genre-wri...,E,The Big Short
20905,864335,2015-12-17-090650,That film was supposed to be a sort-of comedic...,Adam McKay,"['Q350405', 'Q41538972']",2015-12-17 16:41:24,2,"[['Adam McKay', '0.6969'], ['None', '0.1961'],...",['http://www.edmontonsun.com/2015/12/17/direct...,E,The Big Short
25312,1048072,2015-12-02-055009,"In all fairness, he (Clooney) outbid me on Arg...",Brad Pitt,"['Q35332', 'Q373912']",2015-12-02 11:43:04,1,"[['Brad Pitt', '0.8497'], ['None', '0.142'], [...",['http://tribune.com.pk/story/1002292/brad-pit...,E,The Big Short
39046,1615344,2015-12-31-025532,I read The Big Short in one night and thought ...,Adam McKay,"['Q350405', 'Q41538972']",2015-12-31 03:39:00,1,"[['Adam McKay', '0.6069'], ['None', '0.2658'],...",['http://au.news.yahoo.com/thewest/entertainme...,E,The Big Short


In [7]:
df_total = pd.merge(quotes, moviename, on=["primaryTitle"])
#df_total.dtypes

## Part1: Sentiment polarity

In [8]:
afn = Afinn()

nlp = en_core_web_sm.load()

punctuations = string.punctuation
#stopwords = list(STOP_WORDS)

parser = nlp
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in punctuations ]
    tokens = " ".join([i for i in tokens])
    return tokens

def count_words(s):
    return len(s.split())

def sentiment(s):
    return afn.score(s)

In [9]:
df_total["text_clean"] = df_total["quotation"].progress_apply(spacy_tokenizer)

df_total["text"] = df_total["text_clean"].astype('str')

  0%|          | 0/330 [00:00<?, ?it/s]

In [10]:
df_total["sentiment"] = df_total["text"].progress_apply(sentiment)
df_total["word_counts"] = df_total["text"].progress_apply(count_words)

  0%|          | 0/330 [00:00<?, ?it/s]

  0%|          | 0/330 [00:00<?, ?it/s]

In [11]:
df_total["word_counts"] = df_total[df_total["word_counts"]>5]

In [12]:
df_new = df_total[["text", "sentiment", "word_counts", "speaker", "date", "startYear", "primaryTitle", "genres", "Total Gross"]]
df_new = df_new.rename(columns={'date': 'quote_date', 'startYear': 'release_year', 'primaryTitle': 'movie_name'})
                                
df_ready = df_new.drop_duplicates()
df_ready.head()

,text,sentiment,word_counts,speaker,quote_date,release_year,movie_name,genres,Total Gross
0,i think ira glass be no dope i'll read that an...,6.0,638594,Adam McKay,2015-12-21 08:00:00,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
10,well definitely paul rudd and we 've get andre...,3.0,767877,Peyton Reed,2015-12-09 03:21:58,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
20,that film be suppose to be a sort of comedic p...,-1.0,864335,Adam McKay,2015-12-17 16:41:24,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
30,in all fairness he clooney outbid i on argo bu...,7.0,1048072,Brad Pitt,2015-12-02 11:43:04,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
40,i read the big short in one night and think ho...,-2.0,1615344,Adam McKay,2015-12-31 03:39:00,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"


## Part2: Semantic Analysis

**Analyse how people use psychology related vocabularies to talk about certain movies**

Alternative way:
We can analyse Ekamn's universal emotions from quotes: 
["fear", "anger", "joy", "sadness", "contentment", "disgust", "surprise"]

In [19]:
from empath import Empath
lexicon = Empath()

results=[]
for i in df_ready["text"]:
    experience = lexicon.analyze(i,categories = ["warmth", "fun", "emotional", "disappointment", "love", "hate", "cheerfulness", "nervousness", "suffering"], normalize = True)
    results.append(experience)

records = pd.DataFrame() 
for i in results:
    dataframe = pd.DataFrame.from_dict(i, orient="index")
    new_data = dataframe.transpose()
    records = records.append(new_data)

records.head()

,warmth,fun,emotional,disappointment,love,hate,cheerfulness,nervousness,suffering
0,0.0,0.000000,0.011765,0.0,0.000000,0.0,0.0,0.0,0.0
0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
0,0.0,0.025641,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
0,0.0,0.000000,0.000000,0.0,0.019608,0.0,0.0,0.0,0.0
0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [20]:
records['tmp'] = 1
df_ready['tmp'] = 1

df_update = pd.merge(records, df_ready, on=['tmp'])
df_final = df_update.drop('tmp', axis=1)
df_final.head()

/var/folders/_j/mtdz_6x90ks158_q3rnzw_s00000gq/T/ipykernel_5040/3456006716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready['tmp'] = 1


,warmth,fun,emotional,disappointment,love,hate,cheerfulness,nervousness,suffering,text,sentiment,word_counts,speaker,quote_date,release_year,movie_name,genres,Total Gross
0,0.0,0.0,0.011765,0.0,0.0,0.0,0.0,0.0,0.0,i think ira glass be no dope i'll read that an...,6.0,638594,Adam McKay,2015-12-21 08:00:00,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
1,0.0,0.0,0.011765,0.0,0.0,0.0,0.0,0.0,0.0,well definitely paul rudd and we 've get andre...,3.0,767877,Peyton Reed,2015-12-09 03:21:58,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
2,0.0,0.0,0.011765,0.0,0.0,0.0,0.0,0.0,0.0,that film be suppose to be a sort of comedic p...,-1.0,864335,Adam McKay,2015-12-17 16:41:24,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
3,0.0,0.0,0.011765,0.0,0.0,0.0,0.0,0.0,0.0,in all fairness he clooney outbid i on argo bu...,7.0,1048072,Brad Pitt,2015-12-02 11:43:04,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"
4,0.0,0.0,0.011765,0.0,0.0,0.0,0.0,0.0,0.0,i read the big short in one night and think ho...,-2.0,1615344,Adam McKay,2015-12-31 03:39:00,2015,The Big Short,"Biography,Comedy,Drama","$70,259,870"


## Part3: Topic detection

**to check what are the top words related to certain movies**

In [24]:
def extract_vectors(s):
    if s == "":
        return None
    tokens = nlp(s)
    vectors = [word.vector for word in tokens]
    return np.mean(vectors, axis=0)

In [25]:
df_ready["vectors"] = df_ready["text"].progress_apply(extract_vectors)

  0%|          | 0/33 [00:00<?, ?it/s]

/var/folders/_j/mtdz_6x90ks158_q3rnzw_s00000gq/T/ipykernel_5040/3103315908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready["vectors"] = df_ready["text"].progress_apply(extract_vectors)


In [40]:
#TO DO: need to clean stopwords here 
#TO DO: find dominant topic for each text 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

all_text = pd.concat([df_ready["text"]])
#all_text
vectorizer = TfidfVectorizer()
replies_vectorized = vectorizer.fit_transform(all_text)

NUM_TOPICS = 25

lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
topics = lda.fit_transform(replies_vectorized)

def selected_topics(model, vectorizer, top_n=5):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 
selected_topics(lda, vectorizer)

np.mean(np.max(topics, axis=1)>0.1)

topics_ids = np.argmax(topics, axis=1)
topics_per_text = np.split(topics_ids, 3)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
Topic 0:
[('same', 0.30104936987486924), ('character', 0.29074878688050804), ('within', 0.28281727503245113), ('work', 0.28114359597342914), ('his', 0.273353799545776)]
Topic 1:
[('be', 2.1806636298527216), ('to', 1.5650870538676913), ('and', 1.44343946430863), ('the', 1.0991967754119223), ('we', 0.9966677628433787)]
Topic 2:
[('character', 0.2990744347019775), ('agent', 0.2913797327034201), ('be', 0.29033828645424675), ('under', 0.28733048994289495), ('and', 0.28599118833236803)]
Topic 3:
[('fairness', 0.2868556045762726), ('film', 0.2789117813599445), ('present', 0.278223629475066), ('the', 0.27699393723975035), ('couple', 0.27674706343027095)]
Topic 4:
[('get', 2.1466214516885844), ('the', 1.90952